In [1]:
import wrds
import pandas as pd
import numpy as np
db = wrds.Connection(wrds_username='jhirschi')

Loading library list...
Done


In [52]:
db.list_libraries()

['wrds_lib_internal',
 'ppubsamp_d4d_new',
 'ktmine',
 'levin',
 'lspdsamp',
 'ciqsamp_pplintel_new',
 'ciqsamp_common_new',
 'contrib_char_returns_new',
 'taq',
 'secsamp_all_new',
 'compd',
 'compgd',
 'preqsamp_all_new',
 'compbd',
 'compsegd',
 'crspa',
 'reprisk',
 'rpnasamp',
 'secsamp',
 'dmef',
 'taqmsamp',
 'snlsamp',
 'sustsamp',
 'toyo',
 'taqm_2017',
 'trsdcgs',
 'twoiq',
 'wcai',
 'wqa',
 'wrdsapps',
 'wrdsrpts',
 'wappsamp',
 'taqmsec',
 'taqm_2019',
 'trws',
 'tr_13f',
 'bank',
 'blab',
 'compmcur',
 'ftsesamp',
 'gmi',
 'hbase',
 'ifgr',
 'ims',
 'cisdm',
 'issm',
 'nastraq',
 'pacap',
 'ppublica',
 'public',
 'rent',
 'snl',
 'sprat',
 'sustain',
 'centris',
 'emdb',
 'eureka',
 'evts',
 'govpx',
 'gsi',
 'govpxsmp',
 'hbsamp',
 'block_all_new',
 'fisd',
 'etfgsamp',
 'infogroupsamp_residential',
 'ktmine_patents_samp',
 'factset',
 'wrdssec',
 'infogroupsamp_business',
 'compm',
 'ims_obp_trial',
 'ifgrsamp',
 'imssamp',
 'compseg',
 'ravenpack_trial',
 'iri',
 'ktsam

In [4]:
db.list_tables(library='ibes')

['igaact',
 'igachg',
 'igactryactdata',
 'igactryestdata',
 'igaebgdates',
 'igaest',
 'igaindactdata',
 'igaindestdata',
 'igainfo',
 'igarussactdata',
 'igarussestdata',
 'igaspactdata',
 'igaspestdata',
 'igatopixactdata',
 'igatopixestdata',
 'act_epsint',
 'act_epsus',
 'actg',
 'actgu',
 'actpsum_epsint',
 'actpsum_epsus',
 'actpsumu_epsint',
 'actpsumu_epsus',
 'actpsumu_xepsint',
 'actpsumu_xepsus',
 'actpsum_xepsint',
 'actpsum_xepsus',
 'actu_epsint',
 'actu_epsus',
 'actu_xepsint',
 'actu_xepsus',
 'act_xepsint',
 'act_xepsus',
 'adj',
 'adjsum',
 'cfnsum',
 'curr',
 'currnew',
 'det_epsint',
 'det_epsus',
 'detsecd_sepint',
 'detsecd_xsepint',
 'detu_epsint',
 'detu_epsus',
 'detusecd_sepint',
 'detusecd_xsepint',
 'detu_xepsint',
 'detu_xepsus',
 'det_xepsint',
 'det_xepsus',
 'eurx',
 'exc_epsint',
 'exc_epsus',
 'excsecd_sepint',
 'excsecd_xsepint',
 'excu_epsint',
 'excu_epsus',
 'excusecd_sepint',
 'excusecd_xsepint',
 'excu_xepsint',
 'excu_xepsus',
 'exc_xepsint',
 

In [5]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='ibes')
for lib in libs:
    try:
        db.get_table(library='ibes', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass

Success!! This table works:  act_epsint
Success!! This table works:  act_epsus
Success!! This table works:  actg
Success!! This table works:  actgu
Success!! This table works:  actpsum_epsint
Success!! This table works:  actpsum_epsus
Success!! This table works:  actpsumu_epsint
Success!! This table works:  actpsumu_epsus
Success!! This table works:  actpsumu_xepsint
Success!! This table works:  actpsumu_xepsus
Success!! This table works:  actpsum_xepsint
Success!! This table works:  actpsum_xepsus
Success!! This table works:  actu_epsint
Success!! This table works:  actu_epsus
Success!! This table works:  actu_xepsint
Success!! This table works:  actu_xepsus
Success!! This table works:  act_xepsint
Success!! This table works:  act_xepsus
Success!! This table works:  adj
Success!! This table works:  adjsum
Success!! This table works:  cfnsum
Success!! This table works:  curr
Success!! This table works:  currnew
Success!! This table works:  det_epsint
Success!! This table works:  det_ep

In [2]:
db.list_tables(library='comp')

['filings',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'g_chars',
 'g_co_aaudit',
 'g_co_adesind',
 'g_co_afnd1',
 'g_co_afnd2',
 'g_co_afnddc1',
 'g_co_afnddc2',
 'g_co_afntind1',
 'g_co_afntind2',
 'g_co_ainvval',
 'g_co_hgic',
 'g_co_iaudit',
 'g_co_idesind',
 'g_co_ifndq',
 'g_co_ifndsa',
 'g_co_ifndytd',
 'g_co_ifntq',
 'g_co_ifntsa',
 'g_co_ifntytd',
 'g_co_industry',
 'g_co_ipcd',
 'g_company',
 'g_co_offtitl',
 'g_currency',
 'g_ecind_desc',
 'g_ecind_mth',
 'g_exrt_dly',
 'g_exrt_mth',
 'g_funda',
 'g_funda_fncd',
 'g_fundq',
 'g_fundq_fncd',
 'g_idxcst_his',
 'g_idx_daily',
 'g_idx_index',
 'g_idx_mth',
 'g_names',
 'g_names_ix',
 'g_names_ix_cst',
 'g_namesq',
 'g_sec_adesind',
 'g_sec_adjfact',
 'g_sec_af

In [3]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='comp')
for lib in libs:
    try:
        db.get_table(library='comp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


Success!! This table works:  bank_aacctchg
Success!! This table works:  bank_adesind
Success!! This table works:  bank_afnd1
Success!! This table works:  bank_afnd2
Success!! This table works:  bank_afnddc1
Success!! This table works:  bank_afnddc2
Success!! This table works:  bank_afntind
Success!! This table works:  bank_funda
Success!! This table works:  bank_funda_fncd
Success!! This table works:  bank_fundq
Success!! This table works:  bank_fundq_fncd
Success!! This table works:  bank_iacctchg
Success!! This table works:  bank_idesind
Success!! This table works:  bank_ifndq
Success!! This table works:  bank_ifndytd
Success!! This table works:  bank_ifntq
Success!! This table works:  bank_ifntytd
Success!! This table works:  bank_names
Success!! This table works:  bank_namesq
Success!! This table works:  g_chars
Success!! This table works:  g_co_aaudit
Success!! This table works:  g_co_adesind
Success!! This table works:  g_co_afnd1
Success!! This table works:  g_co_afnd2
Success!!

Success!! This table works:  r_inactvcd
Success!! This table works:  r_incstats
Success!! This table works:  r_indfmt
Success!! This table works:  r_indsec
Success!! This table works:  r_invval
Success!! This table works:  r_issuetyp
Success!! This table works:  r_majidxcl
Success!! This table works:  r_mic_codes
Success!! This table works:  r_naiccd
Success!! This table works:  r_notetype
Success!! This table works:  r_ntsubtype
Success!! This table works:  r_offcrso
Success!! This table works:  r_ogmethod
Success!! This table works:  r_opinions
Success!! This table works:  r_prc_stat
Success!! This table works:  r_qsrcdoc
Success!! This table works:  r_secannfn
Success!! This table works:  r_sec_stat
Success!! This table works:  r_sectors
Success!! This table works:  r_siccd
Success!! This table works:  r_spiicd
Success!! This table works:  r_spmicd
Success!! This table works:  r_statalrt
Success!! This table works:  r_states
Success!! This table works:  r_stko
Success!! This table w

In [20]:
#a quick way to find out which schemas we actually have access to in WRDS CRSP
libs = db.list_tables(library='crsp')
for lib in libs:
    try:
        db.get_table(library='crsp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


Success!! This table works:  contact_info
Success!! This table works:  crsp_cik_map
Success!! This table works:  crsp_portno_map
Success!! This table works:  daily_nav
Success!! This table works:  daily_nav_ret
Success!! This table works:  daily_returns
Success!! This table works:  dividends
Success!! This table works:  front_load
Success!! This table works:  front_load_det
Success!! This table works:  front_load_grp
Success!! This table works:  fund_fees
Success!! This table works:  fund_flows
Success!! This table works:  fund_hdr
Success!! This table works:  fund_hdr_hist
Success!! This table works:  fund_names
Success!! This table works:  fund_style
Success!! This table works:  fund_summary
Success!! This table works:  fund_summary2
Success!! This table works:  holdings
Success!! This table works:  mfdbname
Success!! This table works:  monthly_nav
Success!! This table works:  monthly_returns
Success!! This table works:  monthly_tna
Success!! This table works:  monthly_tna_ret_nav
Su

In [25]:
db.describe_table(library='crsp', table='portnomap')

Approximately 73407 rows in crsp.portnomap.


,name,nullable,type
0,crsp_fundno,True,DOUBLE PRECISION
1,crsp_portno,True,DOUBLE PRECISION
2,begdt,True,DATE
3,enddt,True,DATE
4,cusip8,True,VARCHAR(8)
5,crsp_cl_grp,True,DOUBLE PRECISION
6,fund_name,True,VARCHAR(140)
7,ticker,True,VARCHAR(5)
8,ncusip,True,VARCHAR(9)
9,first_offer_dt,True,DATE


In [28]:
db.get_table(library='crsp', table='stocknames', obs=10)

,permno,permco,namedt,nameenddt,cusip,ncusip,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
0,10000.0,7952.0,1986-01-07,1987-06-11,68391610,68391610,OMFGA,OPTIMUM MANUFACTURING INC,3.0,3.0,3990.0,10.0,A,1986-01-31,1987-06-30,2.0
1,10001.0,7953.0,1986-01-09,1993-11-21,36720410,39040610,GFGC,GREAT FALLS GAS CO,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
2,10001.0,7953.0,1993-11-22,2008-02-04,36720410,29274A10,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
3,10001.0,7953.0,2008-02-05,2009-08-03,36720410,29274A20,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
4,10001.0,7953.0,2009-08-04,2009-12-17,36720410,29269V10,EGAS,ENERGY INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
5,10001.0,7953.0,2009-12-18,2010-07-08,36720410,29269V10,EGAS,ENERGY INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
6,10001.0,7953.0,2010-07-09,2017-08-03,36720410,36720410,EGAS,GAS NATURAL INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
7,10002.0,7954.0,1986-01-10,1993-09-29,05978R10,60740110,MBNC,MOBILE NATIONAL CORP,3.0,3.0,6710.0,10.0,None,1986-01-31,2013-02-28,2.0
8,10002.0,7954.0,1993-09-30,2002-05-14,05978R10,83623410,SABC,SOUTH ALABAMA BANCORPORATION IN,3.0,3.0,6710.0,11.0,None,1986-01-31,2013-02-28,2.0
9,10002.0,7954.0,2002-05-15,2013-02-15,05978R10,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,3.0,3.0,6020.0,11.0,None,1986-01-31,2013-02-28,2.0


In [31]:
db.raw_sql('''select * from crsp.stocknames where ticker = 'MSFT' ''')

,permno,permco,namedt,nameenddt,cusip,ncusip,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
0,10107.0,8048.0,1986-03-13,2019-12-31,59491810,59491810,MSFT,MICROSOFT CORP,3.0,3.0,7370.0,11.0,None,1986-03-31,2019-12-31,2.0


In [34]:
stk_names = db.raw_sql(''' select permno, permco, cusip, ncusip, ticker, comnam, exchcd from crsp.stocknames where end_date = '2019-12-31' ''')

In [35]:
stk_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15003 entries, 0 to 15002
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   permno  15003 non-null  float64
 1   permco  15003 non-null  float64
 2   cusip   15003 non-null  object 
 3   ncusip  14544 non-null  object 
 4   ticker  14389 non-null  object 
 5   comnam  15003 non-null  object 
 6   exchcd  15003 non-null  float64
dtypes: float64(3), object(4)
memory usage: 820.6+ KB


In [36]:
stk_names.loc[stk_names['ticker'].isnull()]

,permno,permco,cusip,ncusip,ticker,comnam,exchcd
24,10051.0,7999.0,41043F20,41043F20,None,HANGER INC,0.0
26,10065.0,20023.0,00621210,None,None,ADAMS EXPRESS CO,1.0
36,10145.0,22168.0,43851610,None,None,ALLIED CHEMICAL & DYE CORP,1.0
37,10145.0,22168.0,43851610,None,None,ALLIED CHEMICAL CORP,1.0
46,10180.0,9743.0,00972810,00972810,None,AKORN INC,0.0
...,...,...,...,...,...,...,...
14463,92802.0,53091.0,15234Q20,15234Q20,None,CENTRAIS ELETRICAS BRASILEIRAS,0.0
14580,92893.0,53123.0,82835P10,82835P10,None,SILVERCORP METALS INC,0.0
14856,93263.0,53346.0,15118V20,15118V20,None,CELSIUS HOLDINGS INC,0.0
14963,93369.0,53403.0,74939710,00438V10,None,ACCRETIVE HEALTH INC,0.0


In [37]:
stk_names[['ticker', 'exchcd']].groupby('exchcd').count()

,ticker
exchcd,
-2.0,8
-1.0,40
0.0,0
1.0,4455
2.0,1458
3.0,5806
4.0,2233
5.0,363
6.0,1


In [48]:
db.describe_table(library='crsp', table='dsf')

Approximately 96285900 rows in crsp.dsf.


,name,nullable,type
0,cusip,True,VARCHAR
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


In [7]:
db.describe_table(library='comp', table='fundq').name.tolist()

Approximately 1817440 rows in comp.fundq.


['gvkey',
 'datadate',
 'fyearq',
 'fqtr',
 'fyr',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchgq',
 'acctstdq',
 'adrrq',
 'ajexq',
 'ajpq',
 'bsprq',
 'compstq',
 'curcdq',
 'curncdq',
 'currtrq',
 'curuscnq',
 'datacqtr',
 'datafqtr',
 'finalq',
 'ogmq',
 'rp',
 'scfq',
 'srcq',
 'staltq',
 'updq',
 'apdedateq',
 'fdateq',
 'pdateq',
 'rdq',
 'acchgq',
 'acomincq',
 'acoq',
 'actq',
 'altoq',
 'ancq',
 'anoq',
 'aociderglq',
 'aociotherq',
 'aocipenq',
 'aocisecglq',
 'aol2q',
 'aoq',
 'apq',
 'aqaq',
 'aqdq',
 'aqepsq',
 'aqpl1q',
 'aqpq',
 'arcedq',
 'arceepsq',
 'arceq',
 'atq',
 'aul3q',
 'billexceq',
 'capr1q',
 'capr2q',
 'capr3q',
 'capsftq',
 'capsq',
 'ceiexbillq',
 'ceqq',
 'cheq',
 'chq',
 'cibegniq',
 'cicurrq',
 'ciderglq',
 'cimiiq',
 'ciotherq',
 'cipenq',
 'ciq',
 'cisecglq',
 'citotalq',
 'cogsq',
 'csh12q',
 'cshfd12',
 'cshfdq',
 'cshiq',
 'cshopq',
 'cshoq',
 'cshprq',
 'cstkcvq',
 'cstkeq',
 'cstkq',
 'dcomq',
 'dd1q',
 'derac

In [50]:
4455+1458+5806+2233

13952